# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [12]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd

recipes = pd.read_csv("recipes.tsv", sep="\t")
recipe_tags = pd.read_csv("recipe-tags.tsv", sep="\t")

recipes.head(), recipe_tags.head()

(        recipe_slug      recipe_title  \
 0           falafel           Falafel   
 1        spamburger        Spamburger   
 2  bacon-fried-rice  Bacon Fried Rice   
 3   chicken-fingers   Chicken Fingers   
 4       apple-crisp       Apple Crisp   
 
                                  recipe_introduction  
 0  Falafel is a popular Middle Eastern dish made ...  
 1  Spamburger is a type of hamburger that is made...  
 2  Bacon fried rice is a savory and satisfying di...  
 3  Chicken fingers are a popular dish made from c...  
 4  Apple crisp is a classic dessert made with bak...  ,
    recipe_slug recipe_tag
 0  spam-musubi   hawaiian
 1  spam-musubi       nori
 2  spam-musubi    onthego
 3  spam-musubi       rice
 4  spam-musubi      snack)

In [13]:
# --- Build tag matrix: rows=recipe_slug, columns=tags, values=0/1
tag_matrix = pd.crosstab(recipe_tags["recipe_slug"], recipe_tags["recipe_tag"]).astype(int)

# Align to the 100 recipes in recipes.tsv
tag_matrix = tag_matrix.reindex(recipes["recipe_slug"]).fillna(0).astype(int)

# --- Persona preference weights
pos_tags = [
    "dessert","chocolate","chocolatechip","brownies","cake","cookies",
    "pastry","bakery","baking","sweet","berries","apple","blueberry",
    "strawberry","vanilla","cinnamon","brownsugar","buttery","breakfastpastry"
]
neg_tags = ["seafood","spicy","chilioil","chilipowder","sushi","nori","shrimp","garlic","onion","mushroom","tofu","vegan"]

pos_tags = [t for t in pos_tags if t in tag_matrix.columns]
neg_tags = [t for t in neg_tags if t in tag_matrix.columns]

np.random.seed(42)
score = (
    1.5 * tag_matrix[pos_tags].sum(axis=1)
    - 1.0 * tag_matrix[neg_tags].sum(axis=1)
    + np.random.normal(0, 0.5, size=len(tag_matrix))
)

# Convert score into 5-level raw ratings 1..5 using quintiles
rating_raw = pd.qcut(score.rank(method="first"), 5, labels=[1,2,3,4,5]).astype(int)

# Ensure extremes: bottom 10 => 1, top 10 => 5 (this guarantees >=10 zeros and >=10 ones after mapping)
sorted_idx = score.sort_values().index
rating_raw.loc[sorted_idx[:10]] = 1
rating_raw.loc[sorted_idx[-10:]] = 5

ratings = pd.DataFrame({
    "recipe_slug": recipes["recipe_slug"],
    "rating_raw": rating_raw.loc[recipes["recipe_slug"]].values
})
ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25  # map 1..5 -> 0..1

# sanity checks
print("Counts of ratings:", ratings["rating"].value_counts().sort_index().to_dict())
print("Num zeros:", (ratings["rating"] == 0).sum(), " Num ones:", (ratings["rating"] == 1).sum())

ratings_out = ratings[["recipe_slug","rating"]]
ratings_out.to_csv("ratings.tsv", sep="\t", index=False)
ratings_out.head()

Counts of ratings: {0.0: 20, 0.25: 20, 0.5: 20, 0.75: 20, 1.0: 20}
Num zeros: 20  Num ones: 20


,recipe_slug,rating
0,falafel,0.25
1,spamburger,0.25
2,bacon-fried-rice,0.50
3,chicken-fingers,0.75
4,apple-crisp,0.75


In [14]:
# Start from tag_matrix again (0/1)
features = tag_matrix.copy()

# Add bias column (hard-coded to 1)
features.insert(0, "bias", 1)

# Put recipe_slug as a column (not index)
features.insert(0, "recipe_slug", features.index)

# Align to ratings order (critical!)
features = features.set_index("recipe_slug").loc[ratings_out["recipe_slug"]].reset_index()

features.to_csv("features.tsv", sep="\t", index=False)
features.head()

recipe_tag,recipe_slug,bias,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,spamburger,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bacon-fried-rice,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chicken-fingers,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple-crisp,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
from sklearn.linear_model import Ridge

X = features.drop(columns=["recipe_slug"]).to_numpy(dtype=float)
y = ratings_out["rating"].to_numpy(dtype=float)

ridge = Ridge(alpha=1.0, fit_intercept=False)
ridge.fit(X, y)

coef = ridge.coef_
colnames = features.columns.drop("recipe_slug")

model = pd.DataFrame({
    "recipe_tag": colnames,
    "coefficient": coef
})
model.to_csv("model.tsv", sep="\t", index=False)
model.head()

,recipe_tag,coefficient
0,bias,0.344368
1,alfredo,0.027877
2,almond,-0.009871
3,american,0.000001
4,appetizer,-0.002693


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [16]:
# YOUR CHANGES HERE

from sklearn.linear_model import Ridge

X = features.drop(columns=["recipe_slug"]).to_numpy(dtype=float)
y = ratings_out["rating"].to_numpy(dtype=float)

ridge = Ridge(alpha=1.0, fit_intercept=False)
ridge.fit(X, y)

coef = ridge.coef_
colnames = features.columns.drop("recipe_slug")

model = pd.DataFrame({
    "recipe_tag": colnames,
    "coefficient": coef
})
model.to_csv("model.tsv", sep="\t", index=False)
model.head()

,recipe_tag,coefficient
0,bias,0.344368
1,alfredo,0.027877
2,almond,-0.009871
3,american,0.000001
4,appetizer,-0.002693


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [17]:
# YOUR CHANGES HERE

score_est = X @ coef

estimates = pd.DataFrame({
    "recipe_slug": features["recipe_slug"],
    "score_estimate": score_est
})
estimates.to_csv("estimates.tsv", sep="\t", index=False)
estimates.head()

,recipe_slug,score_estimate
0,falafel,0.217890
1,spamburger,0.243683
2,bacon-fried-rice,0.468329
3,chicken-fingers,0.669914
4,apple-crisp,0.731580


Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [18]:
# YOUR CHANGES HERE

alpha_ucb = 2.0
d = X.shape[1]

A = X.T @ X + np.eye(d)
A_inv = np.linalg.inv(A)

# variance term for each row
var = np.einsum("ij,jk,ik->i", X, A_inv, X)

score_bound = score_est + alpha_ucb * np.sqrt(var)

bounds = pd.DataFrame({
    "recipe_slug": features["recipe_slug"],
    "score_bound": score_bound
})
bounds.to_csv("bounds.tsv", sep="\t", index=False)
bounds.head()

,recipe_slug,score_bound
0,falafel,1.984146
1,spamburger,2.138019
2,bacon-fried-rice,2.334486
3,chicken-fingers,2.464029
4,apple-crisp,2.507922


Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [19]:
alpha_ucb = 2.0
d = X.shape[1]

A = X.T @ X + np.eye(d)

# Compute score_est using your ridge coefficients
score_est = X @ coef

# variance term: x^T A^{-1} x, computed via solving A v = x
V = np.linalg.solve(A, X.T)          # shape: (d, n)
var = np.einsum("ij,ji->i", X, V)    # diagonal of X A^{-1} X^T

score_bound = score_est + alpha_ucb * np.sqrt(var)

bounds = pd.DataFrame({
    "recipe_slug": features["recipe_slug"],
    "score_bound": score_bound
})
bounds.to_csv("bounds.tsv", sep="\t", index=False)
bounds.head()

,recipe_slug,score_bound
0,falafel,1.984146
1,spamburger,2.138019
2,bacon-fried-rice,2.334486
3,chicken-fingers,2.464029
4,apple-crisp,2.507922


In [20]:
# 1) ratings requirements
print("zeros:", (ratings_out["rating"] == 0).sum(), "ones:", (ratings_out["rating"] == 1).sum())

# 2) ordering match
print("same order:", (features["recipe_slug"].tolist() == ratings_out["recipe_slug"].tolist()))

# 3) file existence (after you run all parts)
import os
for f in ["ratings.tsv","features.tsv","model.tsv","estimates.tsv","bounds.tsv","recommendations.tsv","acknowledgments.txt"]:
    print(f, os.path.exists(f))

zeros: 20 ones: 20
same order: True
ratings.tsv True
features.tsv True
model.tsv True
estimates.tsv True
bounds.tsv True
recommendations.tsv True
acknowledgments.txt True


In [21]:
print(pd.read_csv("ratings.tsv", sep="\t").columns.tolist())
print(pd.read_csv("features.tsv", sep="\t").columns[:5].tolist(), "... total:", pd.read_csv("features.tsv", sep="\t").shape[1])
print(pd.read_csv("model.tsv", sep="\t").columns.tolist())
print(pd.read_csv("estimates.tsv", sep="\t").columns.tolist())
print(pd.read_csv("bounds.tsv", sep="\t").columns.tolist())
print(pd.read_csv("recommendations.tsv", sep="\t").columns.tolist())

['recipe_slug', 'rating']
['recipe_slug', 'bias', 'alfredo', 'almond', 'american'] ... total: 298
['recipe_tag', 'coefficient']
['recipe_slug', 'score_estimate']
['recipe_slug', 'score_bound']
['recipe_slug', 'score_bound', 'reward']


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

Hint: do not remove recipes after each recommendation.
Repeating recommendations is expected.

In [22]:
# YOUR CHANGES HERE

with open("acknowledgments.txt", "w") as f:
    f.write(
        "Used ChatGPT for guidance and code structure for generating features, ridge regression, and LinUCB outputs.\n"
    )

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.